In [1]:
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $file.^.cpinit, cpinit._
import ss.implicits._
import org.apache.spark.sql.functions._
import java.nio.file.Paths

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/10 22:09:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$          , sparkinit._

import $file.$         , pathinit._

import $file.$       , cpinit._

import ss.implicits._

import org.apache.spark.sql.functions._

import java.nio.file.Paths

In [2]:
import pureconfig._
import pureconfig.generic.auto._
import com.relatedsciences.opentargets.etl.pipeline.{EvidencePreparationPipeline, PipelineState, Pipeline}
import com.relatedsciences.opentargets.etl.configuration.Configuration.Config

import pureconfig._

import pureconfig.generic.auto._

import com.relatedsciences.opentargets.etl.pipeline.{EvidenceValidationPipeline, PipelineState, Pipeline}

import com.relatedsciences.opentargets.etl.configuration.Configuration.Config

In [3]:
val config = {
    val c = ConfigSource.file(TEST_RESOURCE_DIR.resolve("config/application.conf")).loadOrThrow[Config]
    c.copy(
        inputDir=REPO_DIR.resolve(c.inputDir).toString,
        outputDir=REPO_DIR.resolve(c.outputDir).toString,
        resourceDir=REPO_DIR.resolve(c.resourceDir).toString,
    )
}

config: Config = Config(
  "",
  "/home/eczech/repos/ot-scoring/src/test/resources/pipeline_test/input",
  "/home/eczech/repos/ot-scoring/src/test/resources/pipeline_test/output",
  "/home/eczech/repos/ot-scoring/src/main/resources/data",
  "WARN",
  "https://raw.githubusercontent.com/opentargets/json_schema/1.6.2/opentargets.json",
  ExternalConfig(
    https://storage.googleapis.com/open-targets-data-releases/19.11/input/mrtarget.data.19.11.yml,
    https://github.com/opentargets/data_pipeline/blob/329ff219f9510d137c7609478b05d358c9195579/mrtarget.es.yml
  ),
  Pipeline(
    PipelineScoring(
      true,
      false,
      false,
      "evidence_ese.json",
      Map(
        "sysbio" -> 0.5,
        "progeny" -> 0.5,
        "europepmc" -> 0.2,
        "expression_atlas" -> 0.2,
        "phenodigm" -> 0.2,
        "slapenrich" -> 0.5,
        "crisp" -> 1.0
      )
    ),
    PipelineEvidence("evidence_raw.json", "gene_ese.json"),
    Map(
      "time" -> PipelineDecoratorConfig(true)

In [4]:
val pipeline = new EvidencePreparationPipeline(ss, config)
import pipeline._

pipeline: EvidenceValidationPipeline = com.relatedsciences.opentargets.etl.pipeline.EvidenceValidationPipeline@ea432fd
import pipeline._

In [5]:
val op = Pipeline
  .Builder(config)
  .start("getEvidenceRawData", () => ss.read.textFile(config.rawEvidencePath))
  .andThen("runEvidenceSchemaValidation", runEvidenceSchemaValidation)
  .andThen("parseEvidenceData", parseEvidenceData)
  .andThen("normalizeTargetIds", normalizeTargetIds)
  .operation

op: com.relatedsciences.opentargets.etl.pipeline.Components.Operation[org.apache.spark.sql.package.DataFrame] = com.relatedsciences.opentargets.etl.pipeline.Decorator$ReferenceDecorator$$anon$4@39fd2782

In [6]:
val df = op.run(new PipelineState).cache()

19/12/10 22:09:17 INFO Decorator$: Executing operation 'normalizeTargetIds'
19/12/10 22:09:17 INFO Decorator$: Executing operation 'parseEvidenceData'
19/12/10 22:09:17 INFO Decorator$: Executing operation 'runEvidenceSchemaValidation'
19/12/10 22:09:17 INFO Decorator$: Executing operation 'getEvidenceRawData'
19/12/10 22:09:19 INFO Decorator$: Operation 'getEvidenceRawData' complete
19/12/10 22:09:19 WARN ScalaReflection: Scala version 2.12.10 cannot get type nullability correctly via reflection, thus Spark cannot add proper input null check for UDF. To avoid this problem, use the typed UDF interfaces instead.


save at SparkPipeline.scala:24

1 / 1

save at SparkPipeline.scala:24

1 / 1

...

1 / 1

csv at EvidencePreparationPipeline.scala:191

1 / 1

19/12/10 22:09:28 INFO Decorator$: Operation 'normalizeTargetIds' complete


df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [access_level: string, disease: struct<id: string, biosample: struct<id: string, name: string> ... 2 more fields> ... 12 more fields]

In [14]:
df.select(df("target_id_type")).distinct.map(_(0).toString).collect()

collect at cmd13.sc:1

1 / 1

collect at cmd13.sc:1

1 / 1

res13: Array[String] = Array("ensembl", "uniprot")

In [21]:
//   "P35354" -> "ENSG00000073756",
//   "P10275" -> "ENSG00000169083"
df.filter($"target.id".isin(Seq("ENSG00000073756", "ENSG00000169083"):_*))
    .select($"target.id").distinct.count()

count at cmd20.sc:2

1 / 1

count at cmd20.sc:2

1 / 1

res20: Long = 2L

In [7]:
df.groupBy("target_id_type").count().show()

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

...

show at cmd6.sc:1

1 / 1

show at cmd6.sc:1

1 / 1

+--------------+-----+
|target_id_type|count|
+--------------+-----+
|       ensembl|  151|
|       uniprot|   40|
+--------------+-----+



In [13]:
df.filter($"target.id".contains("uniprot")).select("target.*").show(10, 100)

show at cmd12.sc:1

1 / 1

+---+--------+---------------------+----------+------------+-----------+-----------+----+
| id|activity|binding_subunit_known|complex_id|target_class|target_name|target_type|tier|
+---+--------+---------------------+----------+------------+-----------+-----------+----+
+---+--------+---------------------+----------+------------+-----------+-----------+----+

